#**Asignación 3 - PIB**
##**Nombres**: Robert Orcasitas, Damian Ospina y María Estupiñan

### *Se importan las librerias que se van a utilizar*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp
import plotly.express as px
!pip install plotly
import plotly.graph_objects as go

*Añadimos los datos encontrados en El Banco Mundial, acerca de las estadísticas económicas de Colombia en las que se muestra como invirtió el PIB en los todos los años desde 2000 a 2020 en las áreas, salud, investigación y desarrollo, militar y educación.*

In [ ]:
di = pd.read_csv('pibest.csv')


años = di['Año'].values
n = np.size(años)


*Hallamos los promedios de cada área de inversión en los 20 años*

In [ ]:
# Promedios en Militar, Salud, Educación, e investigación y desarrollo

promedios = np.array([sum(di['Militar'].values)/ n,
                      sum(di['Salud'].values)/ n,
                      sum(di['Educacion'].values)/ n,
                      sum(di['Investigacion'].values)/ n])

# Datos de cada sector para todos los años
M = di['Militar'].values
S = di['Salud'].values
E = di['Educacion'].values
I = di['Investigacion'].values

print("Promedios: \n", promedios)

Promedios: 
 [3.186e+14 6.470e+08 4.039e+14 2.162e+14]


Ahora se calculó la matriz de covarianza para las 4 variables

In [ ]:
cov = np.zeros((4, 4))
cov[0][0] = sum((M - promedios[0]) ** 2)
cov[1][1] = sum((S - promedios[1]) ** 2)
cov[2][2] = sum((E - promedios[2]) ** 2)
cov[3][3] = sum((I - promedios[3]) ** 2)
cov[0][1] = cov[1][0] = sum((M - promedios[0]) * (S - promedios[1]))
cov[0][2] = cov[2][0] = sum((M - promedios[0]) * (E - promedios[2]))
cov[0][3] = cov[3][0] = sum((M - promedios[0]) * (I - promedios[3]))
cov[1][2] = cov[2][1] = sum((S - promedios[1]) * (E - promedios[2]))
cov[1][3] = cov[3][1] = sum((S - promedios[1]) * (I - promedios[3]))
cov[2][3] = cov[3][2] = sum((E - promedios[2]) * (I - promedios[3]))
np.set_printoptions(precision=3, suppress=True)
print("Matriz de covarianza: cov = \n", cov)

Matriz de covarianza: cov = 
 [[ 9.786e+28 -7.190e+22 -1.356e+28 -3.836e+28]
 [-7.190e+22  5.040e+17  4.282e+22  1.287e+23]
 [-1.356e+28  4.282e+22  3.219e+29  7.700e+28]
 [-3.836e+28  1.287e+23  7.700e+28  1.265e+29]]


Ahora calculamos la matriz de correlación de las 4 variables

In [ ]:
sigmas = np.sqrt([sum((M - promedios[0]) ** 2),
                  sum((S - promedios[1]) ** 2),
                  sum((E - promedios[2]) ** 2),
                  sum((I - promedios[3]) ** 2)])

Sigma = np.outer(sigmas, sigmas)
corr = cov / Sigma
print("\nMatriz de correlación: corr = \n", corr)


[3.128e+14 7.099e+08 5.674e+14 3.557e+14]

Matriz de correlación: corr = 
 [[ 1.    -0.324 -0.076 -0.345]
 [-0.324  1.     0.106  0.51 ]
 [-0.076  0.106  1.     0.382]
 [-0.345  0.51   0.382  1.   ]]


*Graficamos*

In [ ]:
cov_largo = pd.DataFrame(cov, columns=['Militar', 'Salud', 'Educacion', 'Investigacion'],
                         index=['Militar', 'Salud', 'Educacion', 'Investigacion']).stack().reset_index()
cov_largo.columns = ['Area de Inversión 1', 'Area de Inversión 2', 'Covarianza']
cov_largo['Valor'] = cov_largo['Covarianza'].round(2).astype(str)

corr_largo = pd.DataFrame(corr, columns=['Militar', 'Salud', 'Educacion', 'Investigacion'],
                          index=['Militar', 'Salud', 'Educacion', 'Investigacion']).stack().reset_index()
corr_largo.columns = ['Area de Inversión 1', 'Area de Inversión 2', 'Correlación']
corr_largo['Valor'] = corr_largo['Correlación'].round(2).astype(str)

#Gráfico de la matriz de covarianza
fig_cov = go.Figure(data=go.Heatmap(
    z=cov_largo['Covarianza'],
    x=cov_largo['Area de Inversión 1'],
    y=cov_largo['Area de Inversión 2'],
    text=cov_largo['Valor'],
    hoverinfo='text',
    colorscale='Turbo'))

for i, val in enumerate(cov_largo['Valor']):
    fig_cov.add_annotation(
        x=cov_largo['Area de Inversión 1'][i],
        y=cov_largo['Area de Inversión 2'][i],
        text=val,
        showarrow=False,
        font=dict(color="white"))

fig_cov.update_layout(
    title='Matriz de Covarianza',
    xaxis_title='Area de Inversión 1',
    yaxis_title='Area de Inversión 2',
    yaxis=dict(scaleanchor="x", scaleratio=0.5,))

fig_cov.show()

#Gráfico de la matriz de correlación
fig_corr = go.Figure(data=go.Heatmap(
    z=corr_largo['Correlación'],
    x=corr_largo['Area de Inversión 1'],
    y=corr_largo['Area de Inversión 2'],
    text=corr_largo['Valor'],
    hoverinfo='text',
    colorscale='Blugrn'))

for i, val in enumerate(corr_largo['Valor']):
    fig_corr.add_annotation(
        x=corr_largo['Area de Inversión 1'][i],
        y=corr_largo['Area de Inversión 2'][i],
        text=val,
        showarrow=False,
        font=dict(color="white"))

fig_corr.update_layout(
    title='Matriz de Correlación',
    xaxis_title='Area de Inversión 1',
    yaxis_title='Area de Inversión 2',
    yaxis=dict(scaleanchor="x", scaleratio=1,))

fig_corr.show()

*Ahora encontramos los autovalores y autovectores asociados a la matriz de covarianza*

In [ ]:
autovalores, autovectores = np.linalg.eig(cov)
print("Matriz de covarianza: \n", cov)
print("\nAutovalores: \n", autovalores)
print("\nAutovectores: \n", autovectores)
resultados = pd.DataFrame(autovectores, columns=[f'Autovector {i+1}' for i in range(4)])
resultados['Autovalor'] = autovalores

print("\nResultados:\n", resultados)

Matriz de covarianza: 
 [[ 9.78557434e+28 -7.19025585e+22 -1.35615160e+28 -3.83581084e+28]
 [-7.19025585e+22  5.04010370e+17  4.28220424e+22  1.28676429e+23]
 [-1.35615160e+28  4.28220424e+22  3.21907094e+29  7.70034337e+28]
 [-3.83581084e+28  1.28676429e+23  7.70034337e+28  1.26538026e+29]]

Autovalores: 
 [3.51191869e+29 1.29271079e+29 6.58379154e+28 3.56940464e+17]

Autovectores: 
 [[-1.01267014e-01  6.77212596e-01 -7.28785354e-01 -3.68355169e-07]
 [ 2.58669917e-07 -9.35246002e-07 -3.99568861e-07 -1.00000000e+00]
 [ 9.35679028e-01  3.13721068e-01  1.61504944e-01 -1.15906705e-07]
 [ 3.38008502e-01 -6.65554049e-01 -6.65423219e-01  9.75771792e-07]]

Resultados:
    Autovector 1  Autovector 2  Autovector 3  Autovector 4     Autovalor
0 -1.012670e-01  6.772126e-01 -7.287854e-01 -3.683552e-07  3.511919e+29
1  2.586699e-07 -9.352460e-07 -3.995689e-07 -1.000000e+00  1.292711e+29
2  9.356790e-01  3.137211e-01  1.615049e-01 -1.159067e-07  6.583792e+28
3  3.380085e-01 -6.655540e-01 -6.654232e-

*Encontramos la Matriz de transformación que nos lleva de la matriz en la base original a la representación de la matriz en la base de autovalores y autovectores.*

In [ ]:
M = np.linalg.inv(autovectores)
print("\nMatriz de cambio de coordenadas M = \n", M)

# Transformamos los autovectores
V1_transf = np.dot(M, autovectores[:, 0])
V2_transf = np.dot(M, autovectores[:, 1])
V3_transf = np.dot(M, autovectores[:, 2])
V4_transf = np.dot(M, autovectores[:, 3])
print("\nV1 transformado = \n", V1_transf)
print("\nV2 transformado = \n", V2_transf)
print("\nV3 transformado = \n", V3_transf)
print("\nV4 transformado = \n", V4_transf)


Matriz de cambio de coordenadas M = 
 [[-1.01267014e-01  2.58669917e-07  9.35679028e-01  3.38008502e-01]
 [ 6.77212596e-01 -9.35246002e-07  3.13721068e-01 -6.65554049e-01]
 [-7.28785354e-01 -3.99568861e-07  1.61504944e-01 -6.65423219e-01]
 [-3.68355169e-07 -1.00000000e+00 -1.15906705e-07  9.75771792e-07]]

V1 transformado = 
 [ 1.00000000e+00 -2.77555756e-17 -2.77555756e-17  5.29395592e-23]

V2 transformado = 
 [-5.55111512e-17  1.00000000e+00  0.00000000e+00 -5.29395592e-23]

V3 transformado = 
 [ 0.00000000e+00 -1.11022302e-16  1.00000000e+00 -1.58818678e-22]

V4 transformado = 
 [ 2.64697796e-23  1.58818678e-22 -1.05879118e-22  1.00000000e+00]
